In [6]:
%store -r X_train
%store -r y_train
%store -r X_test
%store -r y_test

In [1]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()
session_bucket = session.default_bucket()

In [2]:
from sagemaker import KNN

estimator1 = KNN(
    role=role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    sample_size=50,
    k=3,
    feature_dim=5,
    predictor_type="classifier",
    sagemaker_session=session)

In [4]:
record_set = estimator1.record_set(train=X_train, labels=y_train)

In [5]:
estimator1.fit(record_set)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-18 09:15:52 Starting - Starting the training job...
2021-04-18 09:16:15 Starting - Launching requested ML instancesProfilerReport-1618737351: InProgress
......
2021-04-18 09:17:22 Starting - Preparing the instances for training.........
2021-04-18 09:18:35 Downloading - Downloading input data
2021-04-18 09:18:35 Training - Downloading the training image.....Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/18/2021 09:19:33 INFO 140449387971776] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'index_metric': u'L2', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'feature_dim': u'auto', u'faiss_index_ivf_nlists': u'auto', u'epochs': u'1', u'index_type': u'faiss.Flat', u'_faiss_index_nprobe': u'5', u'_kvstore': u'dist_async', u'_num_kv_servers': u'1', u'mini_batch_size': u'5000'}
[04/18/2021 09:19:33 INFO 140449387971776] Merging


2021-04-18 09:19:43 Uploading - Uploading generated training model
2021-04-18 09:19:43 Completed - Training job completed
Training seconds: 75
Billable seconds: 75


In [7]:
predictor1 = estimator1.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!

In [8]:
X_test

array([0.86968265, 0.86968265, 0.86968265, 0.72413793, 0.72413793])

In [23]:
results1 = predictor1.predict(X_test)
results1[0:3]

[label {
   key: "predicted_label"
   value {
     float64_tensor {
       values: 1.0
     }
   }
 },
 label {
   key: "predicted_label"
   value {
     float64_tensor {
       values: 1.0
     }
   }
 },
 label {
   key: "predicted_label"
   value {
     float64_tensor {
       values: 0.0
     }
   }
 }]

In [1]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter04/knn"

In [2]:
!aws s3 cp tmp/train.io s3://{s3_bucket}/{prefix}/input/train.io

upload: tmp/train.io to s3://sagemaker-cookbook-bucket/chapter04/knn/input/train.io


In [15]:
from sagemaker.inputs import TrainingInput

train_path = f"s3://{s3_bucket}/{prefix}/input/train.io"
train = TrainingInput(train_path, 
                      content_type="application/x-recordio-protobuf")

In [16]:
from sagemaker.estimator import Estimator

estimator2 = Estimator(
    image_uri=estimator1.training_image_uri(),
    role=role,
    instance_count=1,
    instance_type='ml.c5.xlarge',
    sagemaker_session=session)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


In [19]:
estimator2.set_hyperparameters(
    sample_size=50,
    k=3,
    predictor_type="classifier")

In [20]:
estimator2.fit({"train": train})

2021-04-18 10:52:15 Starting - Starting the training job...
2021-04-18 10:52:38 Starting - Launching requested ML instancesProfilerReport-1618743135: InProgress
......
2021-04-18 10:53:44 Starting - Preparing the instances for training......
2021-04-18 10:54:39 Downloading - Downloading input data
2021-04-18 10:54:39 Training - Downloading the training image...
2021-04-18 10:55:16 Uploading - Uploading generated training model.Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/18/2021 10:55:13 INFO 140499858621824] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'index_metric': u'L2', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'_log_level': u'info', u'feature_dim': u'auto', u'faiss_index_ivf_nlists': u'auto', u'epochs': u'1', u'index_type': u'faiss.Flat', u'_faiss_index_nprobe': u'5', u'_kvstore': u'dist_async', u'_num_kv_servers': u'1', u'mini_batch_siz


2021-04-18 10:55:39 Completed - Training job completed
Training seconds: 51
Billable seconds: 51


In [21]:
predictor2 = estimator2.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium')

-------------------!

In [22]:
import sagemaker
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor2.serializer = CSVSerializer()
predictor2.deserializer = JSONDeserializer()

In [28]:
results2 = predictor2.predict(X_test)
results2["predictions"][0:3]

[{'predicted_label': 1.0}, {'predicted_label': 1.0}, {'predicted_label': 0.0}]

In [29]:
predictor1.delete_endpoint()

In [30]:
predictor2.delete_endpoint()